In [40]:
from google import genai
import pandas as pd
import random
import time
from IPython.display import Markdown
import time
from tqdm import trange
GEMINI_API_KEY="***REMOVED***"
client = genai.Client(api_key=GEMINI_API_KEY)


In [41]:
import os

VIDEO_PATH = './tesla-real-world-video-q-a/videos/videos/'
video_names = []

for video in os.listdir(VIDEO_PATH):
    if video.endswith('.mp4'):
        
        video_file = client.files.upload(file=VIDEO_PATH + video)
        print(str(video_file.name))
        video_names.append(str(video_file.name))


files/sfgqtfj6bp6u
files/f1cbp454e62f
files/5eohcd7j2lvp
files/ni1kdrojh1lm
files/oxzxz9xrkwjo
files/2kyt4w9l60yz
files/foqcf688zgnl
files/z20vso5ixy7v
files/9m5r5aic7zsn
files/obngf2wpdk7h
files/oa31k3xe75ec
files/ony0ske6va50
files/zors3oa0cnoh
files/tc88glbqcuaq
files/d2y1u5ts1tn0
files/vr145k1x7t7b
files/gjc73oxvshua
files/3oqj5h7g8n6j
files/khf42kq0mwrb
files/30mww5uqkpcp
files/ds10k9ua3q9x
files/en1jmfptmoxo
files/axemjxso0omm
files/7lfjhxe2l2v0
files/95uvgpz5f5cr
files/gn6dmqitjurw
files/kvwtfxigmw5f
files/q95jzwhpqwcy
files/detrzufql5ki
files/7xnel3lqd761
files/zsdhp45lzfo5
files/mt1z63zdtx2e
files/p24jh94gmbzy
files/wvi0851wgvmr
files/vq0h17nsfevd
files/6srr2jyu2heh
files/8gcsen3y5oqa
files/5sfjx5zf8b7m
files/fd6anopopq
files/ktouon74njdj
files/vrb50nilr6ce
files/ozipusvuimwi
files/30cwiycd9crp
files/ydu2cfddymya
files/w503d9eit4r5
files/1z4mrzn88396
files/pi7c9e0x41n5
files/ms9avm739e7d
files/t5hhy49veq3h
files/7eph4cc0uu07


In [46]:
print(video_names)

['files/sfgqtfj6bp6u', 'files/f1cbp454e62f', 'files/5eohcd7j2lvp', 'files/ni1kdrojh1lm', 'files/oxzxz9xrkwjo', 'files/2kyt4w9l60yz', 'files/foqcf688zgnl', 'files/z20vso5ixy7v', 'files/9m5r5aic7zsn', 'files/obngf2wpdk7h', 'files/oa31k3xe75ec', 'files/ony0ske6va50', 'files/zors3oa0cnoh', 'files/tc88glbqcuaq', 'files/d2y1u5ts1tn0', 'files/vr145k1x7t7b', 'files/gjc73oxvshua', 'files/3oqj5h7g8n6j', 'files/khf42kq0mwrb', 'files/30mww5uqkpcp', 'files/ds10k9ua3q9x', 'files/en1jmfptmoxo', 'files/axemjxso0omm', 'files/7lfjhxe2l2v0', 'files/95uvgpz5f5cr', 'files/gn6dmqitjurw', 'files/kvwtfxigmw5f', 'files/q95jzwhpqwcy', 'files/detrzufql5ki', 'files/7xnel3lqd761', 'files/zsdhp45lzfo5', 'files/mt1z63zdtx2e', 'files/p24jh94gmbzy', 'files/wvi0851wgvmr', 'files/vq0h17nsfevd', 'files/6srr2jyu2heh', 'files/8gcsen3y5oqa', 'files/5sfjx5zf8b7m', 'files/fd6anopopq', 'files/ktouon74njdj', 'files/vrb50nilr6ce', 'files/ozipusvuimwi', 'files/30cwiycd9crp', 'files/ydu2cfddymya', 'files/w503d9eit4r5', 'files/1z4m

In [47]:
questions = pd.read_csv('tesla-real-world-video-q-a/questions.csv')
questions['answers'] = None
questions['freq_map'] = None
questions['entropy'] = None
questions.head()

,id,question,answers,freq_map,entropy
0,1,Was ego doing a legal maneuver if its goal is ...,None,None,None
1,2,Where can ego legally park on this street? A. ...,None,None,None
2,3,What is the best description of the maneuver e...,None,None,None
3,4,Why is ego stopped? A. Judah. B. Traffic Light...,None,None,None
4,5,"What is the blinker state of the oncoming car,...",None,None,None


In [ ]:
def extract_answer(text, depth = 5):
    if depth == 0:
        return "NOT FOUND"
    
    if len(text) == 1:
        return text
    
    temp = client.models.generate_content(
        model='gemini-2.0-flash', contents=f'''You are an answer parser.
        Parse the correct letter choice and include the final answer surrounded by <answer></answer> XML tags. 
        
        ****  ONLY INCLUDE A SINGLE LETTER WITHIN <answer></answer> ****
        
        Prompt:\n{text}
        '''
    ).text
    
    # print(temp)
    
    try:
        cur = temp.split('<answer>')[1].split('</answer>')[0].strip()
        assert(len(cur) == 1)
        assert(cur in ['A', 'B', 'C', 'D'])
        
        if depth != 5:
            print("BAD TEXT", text)
            print("PARSED", cur)
            
        return cur
    
    except Exception as e:
        time.sleep(1)
        print("Not found in", temp)
        return extract_answer(text, depth - 1)
    
    
print(extract_answer('''Here's the breakdown:

* **Ego's Goal:** Turn right.
* **The Lane:** Marked as a right-turn lane by the roundabout sign.

The correct answer is (B). The construction barrels clearly block the right-turn lane, making it illegal for ego to use it.  Ego should wait behind the other vehicles.'''))

B


In [71]:
def parse_question(question_str):
    Q = question_str.split('A.')[0]
    
    A_str = question_str.split(Q)[1]
        
    mp = {}
    
    temp = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
    
    for idx in range(len(temp)):
        cur_choice = temp[idx]
        nxt_choice = temp[idx+1]
        
        if cur_choice + '.' not in A_str:
            print(cur_choice)
            print(A_str)
            break
        
        mp[cur_choice] = A_str.split(cur_choice + '.')[1]
        
        if nxt_choice + '.' in A_str:
            mp[cur_choice] = mp[cur_choice].split(nxt_choice + '.')[0]
    
    return Q, mp

print(parse_question(questions['question'][0]))

E
A. It's legal as the lane is empty. B. It's illegal as the right turn lane is bloacked by construction. C. It's illegal as ego was cutting in other vehicles that were waiting. D. It's legal but the lane ahead is way too narrow for ego to pass.
('Was ego doing a legal maneuver if its goal is to turn right at the intersection? ', {'A': " It's legal as the lane is empty. ", 'B': " It's illegal as the right turn lane is bloacked by construction. ", 'C': " It's illegal as ego was cutting in other vehicles that were waiting. ", 'D': " It's legal but the lane ahead is way too narrow for ego to pass."})


In [73]:
import random

num_shuffles = 1

def answer_question(video_path, question, choices):
    """
    Shuffles answer options while keeping a mapping from fixed labels (A, B, C, D)
    to the original answer keys. Queries the model multiple times, extracts the answer
    with extract_answer(), and uses majority voting over the original options.
    
    Args:
        video_path (str): Path to the video file.
        question (str): The question to ask.
        choices (dict): Dictionary mapping original letters (e.g., 'A', 'B', 'C', 'D')
                        to option text.
                        
    Returns:
        tuple: (majority_vote, vote_counts) where:
            - majority_vote: The original option letter with the most votes.
            - vote_counts: A dictionary counting votes for each original option.
    """
    votes = {}
    fixed_labels = ['A', 'B', 'C', 'D']
    
    for _ in range(num_shuffles):
        # Shuffle the original choices (list of tuples: (original_letter, option_text))
        shuffled_choices = list(choices.items())
        random.shuffle(shuffled_choices)
        
        # Build a mapping from fixed label to the original letter
        mapping = {}  # new label -> original letter
        prompt = 'You have access to these front-view camera images of a car taken at a 0.5 second interval over the past 5 seconds. Imagine you are driving the car.\n\n' + question + '\n\nOptions:\n\n'
        
        
        
        for new_label, (orig_letter, option_text) in zip(fixed_labels, shuffled_choices):
            mapping[new_label] = orig_letter
            prompt += f"{new_label}. {option_text}\n"
                
        # prompt += '\nThink step-by-step about the situation before providing the answer.'
        prompt += '''\nDescribe each scene one by one with respect to the problem statement. Each scene should recieve an analysis, do not group scenes. Then, provide a final answer using your best judgement. If you cannot provide a response, people will get injured. 
        I'll go through each scene again with a detailed breakdown, specifically analyzing where the ego vehicle could legally park.

Example Question: 

Where can ego legally park on this street? A. No parking anywhere. B. next to right curb. C. anywhere. D. next to left curb

Example Response: 

### **Scene 1: Initial View of the Road**
- The road is divided with traffic cones placed along the center.
- No visible parking signs yet.
- The right side has a curb with some parked vehicles.
- The left side appears to be an active lane of traffic with no visible parking spots.

#### **Analysis:**
- The right curb **might** allow parking, but no clear signs are visible.
- The left curb is an active traffic lane, making it unlikely to be a legal parking spot.

---

### **Scene 2: Closer to the Storefronts**
- More traffic cones continue dividing the road.
- Businesses and storefronts are visible on the right side.
- A "No Parking" sign appears near the right curb.
- Some vehicles are parked along the right curb.

#### **Analysis:**
- **Right curb:** The "No Parking" sign suggests that at least part of this section prohibits parking.
- **Left curb:** No indication of parking spots; it remains an active lane.
- **Possible conclusion:** The parked cars may be violating the rule or be in an exempted area beyond the visible sign.

---

### **Scene 3: Passing the Storefronts**
- More storefronts, including a **"DONUTS"** shop, are clearly visible.
- The right curb still has cars parked.
- The "No Parking" sign is still visible in the previous scene.

#### **Analysis:**
- **Right curb:** If the "No Parking" sign applies to the entire stretch, parking is **not allowed** here.
- **Left curb:** No visible parking indicators or parked cars.
- **Potential clarification:** If parking were allowed, there should be a sign indicating that.

---

### **Scene 4: Turning Towards the White Car**
- The camera angle shifts towards the right, showing a parked **white car**.
- The sidewalk and store entrances are clearly visible.
- The "No Parking" sign is not visible in this frame, but we saw it earlier.

#### **Analysis:**
- The **right curb remains questionable**—the sign prohibits parking, yet cars are present.
- The **left curb is still an active lane**, not a designated parking area.

---

### **Scene 5: Moving Past the White Car**
- The vehicle is driving past the **parked white car**.
- The background shows a dimly lit residential area.
- More "No Parking" signs appear in the distance.

#### **Analysis:**
- **Right curb:** The presence of "No Parking" signs suggests that parking is not allowed in these areas.
- **Left curb:** Again, no visible signs allowing parking.

---

### **Scene 6: Near the End of the Sequence**
- The street is darker, with fewer businesses and more residential buildings.
- A **"No Parking" sign is clearly visible** on the right curb.
- Two orange construction cones are placed near the right curb.

#### **Final Analysis:**
- **Right curb:** The visible "No Parking" sign confirms that parking is **not allowed**.
- **Left curb:** No markings or signs suggest it is legal to park there.

---

### **Final Verdict:**
**Answer: A. No Parking Anywhere.**  

- The **right curb** has "No Parking" signs in multiple locations, restricting legal parking.
- The **left curb** remains an active traffic lane and has no indicators allowing parking.
- The **parked cars may be in a zone beyond our field of view where parking is allowed, or they may be parked illegally**.

Thus, the best and safest assumption is that parking is **not permitted anywhere** in this stretch of the road.

Answer: A
        
        
YOUR RESPONSE: 
        '''
        
        
        # Query the model and extract its answer using extract_answer()
        res = query_model(video_path=video_path, prompt=prompt)
        answer_new_label = extract_answer(res)
        
        # Convert the fixed label answer to the original letter
        if answer_new_label in mapping:
            orig_answer = mapping[answer_new_label]
            votes[orig_answer] = votes.get(orig_answer, 0) + 1
        else:
            votes["Invalid"] = votes.get("Invalid", 0) + 1
            
        time.sleep(1)
        
        # Majority voting among valid (original) options
        valid_votes = {k: v for k, v in votes.items() if k in choices}
        if valid_votes:
            majority_vote = max(valid_votes, key=valid_votes.get)
        else:
            majority_vote = "Invalid"
    
    return majority_vote, votes

# Example usage:
print(video_names[0])
print('=' * 10)
print(questions.iloc[0]['question'])
print('=' * 10)
Q, C = parse_question(questions.iloc[0]['question'])
print("Parsed Question:", Q)
print("Parsed Choices:", C)

majority_answer, vote_map = answer_question(video_path=video_names[0], question=Q, choices=C)
print("Majority Answer (original index):", majority_answer)
print("Vote Map (original indices):", vote_map)

files/sfgqtfj6bp6u
Was ego doing a legal maneuver if its goal is to turn right at the intersection? A. It's legal as the lane is empty. B. It's illegal as the right turn lane is bloacked by construction. C. It's illegal as ego was cutting in other vehicles that were waiting. D. It's legal but the lane ahead is way too narrow for ego to pass.
E
A. It's legal as the lane is empty. B. It's illegal as the right turn lane is bloacked by construction. C. It's illegal as ego was cutting in other vehicles that were waiting. D. It's legal but the lane ahead is way too narrow for ego to pass.
Parsed Question: Was ego doing a legal maneuver if its goal is to turn right at the intersection? 
Parsed Choices: {'A': " It's legal as the lane is empty. ", 'B': " It's illegal as the right turn lane is bloacked by construction. ", 'C': " It's illegal as ego was cutting in other vehicles that were waiting. ", 'D': " It's legal but the lane ahead is way too narrow for ego to pass."}
Majority Answer (origin

In [74]:
import math

def calculate_entropy(freq_map):
    total = sum(freq_map.values())
    entropy = -sum((count / total) * math.log2(count / total) for count in freq_map.values() if count > 0)
    return entropy

# Example usage:
frequency_map = {'D': 1, 'A': 3, 'C': 1}
entropy_value = calculate_entropy(frequency_map)
print("Entropy:", entropy_value)

Entropy: 1.3709505944546687


In [75]:
import random

num_shuffles = 1

def answer_question(video_path, question, choices):
    """
    Shuffles answer options while keeping a mapping from fixed labels (A, B, C, D)
    to the original answer keys. Queries the model multiple times, extracts the answer
    with extract_answer(), and uses majority voting over the original options.
    
    Args:
        video_path (str): Path to the video file.
        question (str): The question to ask.
        choices (dict): Dictionary mapping original letters (e.g., 'A', 'B', 'C', 'D')
                        to option text.
                        
    Returns:
        tuple: (majority_vote, vote_counts) where:
            - majority_vote: The original option letter with the most votes.
            - vote_counts: A dictionary counting votes for each original option.
    """
    votes = {}
    fixed_labels = ['A', 'B', 'C', 'D']
    
    for _ in range(num_shuffles):
        # Shuffle the original choices (list of tuples: (original_letter, option_text))
        shuffled_choices = list(choices.items())
        random.shuffle(shuffled_choices)
        
        # Build a mapping from fixed label to the original letter
        mapping = {}  # new label -> original letter
        prompt = question + '\n\nOptions:\n\n'
        for new_label, (orig_letter, option_text) in zip(fixed_labels, shuffled_choices):
            mapping[new_label] = orig_letter
            prompt += f"{new_label}. {option_text}\n"
        
        # Query the model and extract its answer using extract_answer()
        res = query_model(video_path=video_path, prompt=prompt)
        answer_new_label = extract_answer(prompt + '\n\n' + res)
        
        # Convert the fixed label answer to the original letter
        if answer_new_label in mapping:
            orig_answer = mapping[answer_new_label]
            votes[orig_answer] = votes.get(orig_answer, 0) + 1
        else:
            print("VIDEO_PATH IS INVALID ANSER CHOCE", video_path)
            votes["Invalid"] = votes.get("Invalid", 0) + 1
        time.sleep(0.5)
        
    time.sleep(1)
    
    # Majority voting among valid (original) options
    valid_votes = {k: v for k, v in votes.items() if k in choices}
    if valid_votes:
        majority_vote = max(valid_votes, key=valid_votes.get)
    else:
        majority_vote = "Invalid"
    
    return majority_vote, votes

# Example usage:
print(video_names[0])
print('=' * 10)
print(questions.iloc[0]['question'])
print('=' * 10)
Q, C = parse_question(questions.iloc[0]['question'])
print("Parsed Question:", Q)
print("Parsed Choices:", C)

majority_answer, vote_map = answer_question(video_path=video_names[0], question=Q, choices=C)
print("Majority Answer (original index):", majority_answer)
print("Vote Map (original indices):", vote_map)

files/sfgqtfj6bp6u
Was ego doing a legal maneuver if its goal is to turn right at the intersection? A. It's legal as the lane is empty. B. It's illegal as the right turn lane is bloacked by construction. C. It's illegal as ego was cutting in other vehicles that were waiting. D. It's legal but the lane ahead is way too narrow for ego to pass.
E
A. It's legal as the lane is empty. B. It's illegal as the right turn lane is bloacked by construction. C. It's illegal as ego was cutting in other vehicles that were waiting. D. It's legal but the lane ahead is way too narrow for ego to pass.
Parsed Question: Was ego doing a legal maneuver if its goal is to turn right at the intersection? 
Parsed Choices: {'A': " It's legal as the lane is empty. ", 'B': " It's illegal as the right turn lane is bloacked by construction. ", 'C': " It's illegal as ego was cutting in other vehicles that were waiting. ", 'D': " It's legal but the lane ahead is way too narrow for ego to pass."}
Majority Answer (origin

In [76]:
for question_number in trange(0, 50):
    if questions.iloc[question_number]['answers'] is not None:
        continue
    
    raw_video_path = video_names[question_number]
    raw_question = questions.iloc[question_number]['question']
        
    Q, C = parse_question(raw_question)
        
    answered = False
    
    while not answered:
        try: 
            model_ans, freq_mp = answer_question(video_path=raw_video_path, question=Q, choices=C)
            
            questions.at[question_number, 'reasoning'] = model_ans
            questions.at[question_number, 'entropy'] = calculate_entropy(freq_mp)
            questions.at[question_number, 'freq_map'] = freq_mp
            
            answered = True
            
        except Exception as e:
            print(f"Error on question {question_number}: {e}")
        
        print("Rate Limit? Sleeping...")
        time.sleep(10)
        
    time.sleep(1)
    
    

  0%|          | 0/50 [00:00<?, ?it/s]

E
A. It's legal as the lane is empty. B. It's illegal as the right turn lane is bloacked by construction. C. It's illegal as ego was cutting in other vehicles that were waiting. D. It's legal but the lane ahead is way too narrow for ego to pass.
Rate Limit? Sleeping...


  2%|▏         | 1/50 [00:16<13:11, 16.15s/it]

E
A. No parking anywhere. B. next to right curb. C. anywhere. D. next to left curb.
Not found in A. No parking anywhere.

Not found in A. No parking anywhere.

Not found in A. No parking anywhere.

Not found in A. No parking anywhere.

Not found in A. No parking anywhere.

VIDEO_PATH IS INVALID ANSER CHOCE files/f1cbp454e62f
Rate Limit? Sleeping...


  4%|▍         | 2/50 [00:37<15:21, 19.20s/it]

E
A. Lane change to the left and then lane change to the right. B. Lane change to the right and then lane change to the left. C. Staying in a lane which curves to the left and then to the right. D. Staying in a lane which curves to the right and then to the left.
Rate Limit? Sleeping...


  6%|▌         | 3/50 [00:52<13:42, 17.51s/it]

E
A. Judah. B. Traffic Light. C. Someone is crossing the road. D. Construction.
Not found in There is not a definitive answer, but based on my knowledge, a traffic light or construction are the two most probable reasons a car is stopped.

B. Traffic Light.
C. Construction.

Because I do not know which is more likely, I cannot answer.
        

BAD TEXT Why is ego stopped? 

Options:

A.  Someone is crossing the road. 
B.  Traffic Light. 
C.  Construction.
D.  Judah. 


Based on the images provided, it's impossible to definitively determine why the car ("ego") is stopped. Here's why, and what we can infer:

*   **What we see:** A car is stopped on a dark, seemingly rural road. There's another car parked on the side of the road ahead.

*   The car with the headlights on could be stopped due to the presence of that other stationary car. 

Based on the scenario, I cannot determine any of the options are correct. 

PARSED A
Rate Limit? Sleeping...


  6%|▌         | 3/50 [01:10<18:22, 23.45s/it]


KeyboardInterrupt: 

# Parse Questions

In [ ]:
letter_answers = []
for i in questions['reasoning']:
    letter_answers.append(i)
    
print(letter_answers)

temp = []
for i in letter_answers:
    if i not in ['A', 'B', 'C', 'D']:
        temp.append('C')
    else:
        temp.append(i)
        
letter_answers = temp
print(letter_answers)

['C', 'A', 'C', 'Invalid', 'B', 'B', 'A', 'A', 'Invalid', 'Invalid', 'B', 'C', 'C', 'C', 'B', 'A', 'C', 'Invalid', 'A', 'B', 'Invalid', 'Invalid', 'Invalid', 'B', 'Invalid', 'A', 'Invalid', 'A', 'B', 'B', 'C', 'B', 'B', 'B', 'C', 'Invalid', 'A', 'B', 'Invalid', 'B', 'B', 'B', 'Invalid', 'B', 'Invalid', 'A', 'B', 'B', 'A', 'A']
['C', 'A', 'C', 'C', 'B', 'B', 'A', 'A', 'C', 'C', 'B', 'C', 'C', 'C', 'B', 'A', 'C', 'C', 'A', 'B', 'C', 'C', 'C', 'B', 'C', 'A', 'C', 'A', 'B', 'B', 'C', 'B', 'B', 'B', 'C', 'C', 'A', 'B', 'C', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'B', 'B', 'A', 'A']


In [ ]:
dataframe_A, dataframe_B = [], []

idx = 1
while idx <= 251:
    dataframe_A.append(idx)
    if idx-1 < len(letter_answers):
        dataframe_B.append(letter_answers[idx-1])
    else:
        dataframe_B.append('A')
    idx += 1

In [ ]:
answers = pd.DataFrame({'id': dataframe_A, 'answer': dataframe_B})


In [ ]:
answers.to_csv('baseline_answers.csv', index=False)
answers.head()

,id,answer
0,1,C
1,2,A
2,3,A
3,4,A
4,5,A
